In [36]:
import sklearn
print(sklearn.__version__)
#pip install scikit-learn --upgrade

1.0.2


In [37]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [38]:
import pandas as pd
iris = load_iris()

In [39]:
iris_data = iris.data
iris_label = iris.target

In [72]:
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)
iris_df['label'] = iris_label

In [41]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),lablel
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [42]:
X_train , X_test , y_train , y_test = train_test_split(iris_data, iris_label, test_size=0.2, random_state=11)
# random index가 선택된다.

In [43]:
#DecisionTree 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

In [44]:
#학습 수행
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

In [45]:
#학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
pred = dt_clf.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9333


## KFOLD implmentation

In [47]:
from sklearn.model_selection import KFold
import numpy as np

In [49]:
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:', features.shape[0])

붓꽃 데이터 세트 크기: 150


In [55]:
features.shape

(150, 4)

In [67]:
n_iter = 0
for train_index , test_index in kfold.split(features): #Kfold객체의 split()을 호출하면 폴드별 학습용,검즈용 테스트의 데이터포인트 index를 튜플형태로 반환
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter +=1
    
    #반복시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1}, 학습데이터 크기 : {2}, 검증데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)

# iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))



#1 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#5 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


### Stratified K fold

In [74]:
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [81]:
kfold = KFold(n_splits=3)
n_iter = 0
for train_index , test_index in kfold.split(iris_df):
    n_iter +=1 
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 :\n',label_train.value_counts())
    print('검증 레이블 데이터 분포 :\n',label_test.value_counts())
    print('')
    

## 교차 검증: 1
학습 레이블 데이터 분포 :
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    50
Name: label, dtype: int64

## 교차 검증: 2
학습 레이블 데이터 분포 :
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    50
Name: label, dtype: int64

## 교차 검증: 3
학습 레이블 데이터 분포 :
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    50
Name: label, dtype: int64



In [84]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)

for train_index , test_index in skf.split(iris_df,iris_df['label']):  #분기의 기준 라벨 추가 
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 7
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증: 8
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증: 9
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [98]:
## stratified KFold 기반으로 재학습 및 추론
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    #반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f'\n# {n_iter} 교차검증 정확도 : {accuracy}, 학습데이터 크기 : {train_size} , 검증 데이터 크기 : {test_size}')

print('\n'f'평균 검증 정확도 : {np.mean(cv_accuracy)}')



# 1 교차검증 정확도 : 0.98, 학습데이터 크기 : 100 , 검증 데이터 크기 : 50

# 2 교차검증 정확도 : 0.94, 학습데이터 크기 : 100 , 검증 데이터 크기 : 50

# 3 교차검증 정확도 : 0.98, 학습데이터 크기 : 100 , 검증 데이터 크기 : 50

평균 검증 정확도 : 0.9666666666666667


### 교차검증을 보다 편리하게 하는 방법 cross_val_score()

In [102]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data= iris_data.data
label = iris_data.target

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)  #estimator->fit(학습)->예측(pred)->평가(evaluate) 원큐
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### GridSearchCV

150개 -> 120개/30개(test) -> 120개 (80(train):40(validate) fold1 x3)

In [127]:
parameters = {'max_depth' : [1,2,3],
                   'min_samples_split': [2,3]
                   }

In [120]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

iris_data = load_iris()
X_train, X_test , y_train, y_test = train_test_split(iris_data.data, iris_data.target , test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

import pandas as pd  
grid_dtree = GridSearchCV(dtree,param_grid=parameters, cv=3, refit=True)
grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
#scores_df['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000510,3.134972e-04,0.000299,0.000226,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5
1,0.000229,4.769696e-06,0.000126,0.000002,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5
2,0.000229,4.994792e-06,0.000123,0.000008,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3
3,0.000222,9.733398e-07,0.000115,0.000001,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3
4,0.000236,2.213858e-06,0.000197,0.000111,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1
5,0.000327,6.802979e-05,0.000168,0.000035,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1


In [118]:
print('GridSearchCV 최적 파라미터:',grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9750


In [125]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

iris_data = load_iris()
#X_train, X_test , y_train, y_test = train_test_split(iris_data.data, iris_data.target , test_size=0.2, random_state=121)
X_train = iris_data.data
y_train = iris_data.target
dtree = DecisionTreeClassifier()

import pandas as pd  
grid_dtree = GridSearchCV(dtree,param_grid=parameters, cv=3, refit=True)
grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
#scores_df['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000393,0.000126,0.000193,0.000062,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.66,0.66,0.66,0.660000,0.000000,5
1,0.000239,0.000018,0.000132,0.000008,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.66,0.66,0.66,0.660000,0.000000,5
2,0.000237,0.000008,0.000120,0.000002,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.96,0.92,0.92,0.933333,0.018856,3
3,0.000230,0.000002,0.000118,0.000002,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.96,0.92,0.92,0.933333,0.018856,3
4,0.000244,0.000004,0.000116,0.000001,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.98,0.94,1.00,0.973333,0.024944,1
5,0.000357,0.000087,0.000146,0.000014,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.98,0.94,0.96,0.960000,0.016330,2


In [126]:
print('GridSearchCV 최적 파라미터:',grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9733


## 데이터 인코딩

In [129]:
from sklearn.preprocessing import LabelEncoder

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','믹서','믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 된 값:', labels)

인코딩 된 값: [0 1 4 5 3 2 2]


In [130]:
print('인코딩 클래스:', encoder.classes_)

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [131]:
encoder.inverse_transform([5,4,3,2,1,0])

array(['컴퓨터', '전자레인지', '선풍기', '믹서', '냉장고', 'TV'], dtype='<U5')

트리 계열이 아닌 특정 ML 알고리즘에서는 중요도로 인식할 수 있기 때문에 사용상의 주의가 필요하다.

In [134]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','믹서','믹서']
items = np.array(items).reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

<7x6 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [136]:
type(oh_labels)

scipy.sparse._csr.csr_matrix

In [138]:
oh_labels.toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [140]:
oh_labels.shape

(7, 6)

## 타이타닉

In [177]:
train_df = pd.read_csv('../data/titanic/train.csv')
test_df = pd.read_csv('../data/titanic/test.csv')

In [178]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [179]:
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder = encoder.fit(df[feature])
        df[feature] = encoder.transform(df[feature])
     
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [183]:
y = train_df['Survived']
features = train_df.drop('Survived',axis=1)
features = transform_features(features)


In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=11)

In [185]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

#DecisionTree
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test,dt_pred)))


#RandomFOrest
rf_clf.fit(X_train,y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test,rf_pred)))

#LogsiticRegression
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test,lr_pred)))


DecisionTreeClassifier 정확도: 0.7877
RandomForestClassifier 정확도: 0.8547
LogisticRegression 정확도: 0.8659


In [191]:
from sklearn.model_selection import KFold

def exec_KFold(clf,X=X, y=y, folds=5):
    kfold = KFold(n_splits=folds)
    scores = []
    
    for iter, (train_index, test_index) in enumerate(kfold.split(X)): #kfold 는 full-df, skf는 full-df, label
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        
        clf.fit(X_train,y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test,pred)
        scores.append(accuracy)
        print('교차 검증 {} 정확도: {:.4f}'.format(iter,accuracy))
        
    #fold에 대한 평균 계산
    mean_score = np.mean(scores)
    print('평균 정확도: {:.4f}'.format(mean_score))
    
exec_KFold(dt_clf,folds=5)

교차 검증 0 정확도: 0.7542
교차 검증 1 정확도: 0.7809
교차 검증 2 정확도: 0.7865
교차 검증 3 정확도: 0.7697
교차 검증 4 정확도: 0.8202
평균 정확도: 0.7823


In [193]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dt_clf, X,y,cv=5)

for iter, accuracy in enumerate(scores):
    print("교차 검증 {} 정확도 {:.4f}".format(iter,accuracy))
    
print('평균 정확도: {:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도 0.7430
교차 검증 1 정확도 0.7753
교차 검증 2 정확도 0.7921
교차 검증 3 정확도 0.7865
교차 검증 4 정확도 0.8427
평균 정확도: 0.7879


In [200]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,3,5,10],
              'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터 : ',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도 : {:0.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test,dpredictions)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.7992
테스트 세트에서의 DecisionTreeClassifier 정확도 : 0.8715
